# Étude sur les facteurs qui déterminent la santé des enfants aux États-Unis

# Importation des libraries 

In [1]:
import s3fs # pour récupérer les fichiers de l'espace de stockage S3
import pandas as pd # pour la gestion des dataframe
import geopandas as gpd # pour la gestion des données géographiques (fichiers .shp)
import matplotlib.pyplot as plt
import script.clean_data as cd

## Récupération des bases de données

In [2]:
# (depuis le stockage S3 d'un membre du groupe via le dossier public diffusion)
# Stocker les fichiers dans un dossier data/nsch

fs = s3fs.S3FileSystem(client_kwargs={"endpoint_url": "https://minio.lab.sspcloud.fr"})

chemin_nsch = "inacampan/diffusion/Determinants_of_children-s_health/NSCH/"

for year in ["2023", "2022", "2021"]:
    cd.lecture_fichier(fs, f"{chemin_nsch}nsch_{year}e_topical.sas7bdat", f"data/nsch/nsch_{year}e_topical.sas7bdat")

# le fichier est de type database sas
dfs = {}

for year in ["2023", "2022", "2021"]:
    dfs[year] = pd.read_sas(f"data/nsch/nsch_{year}e_topical.sas7bdat", format='sas7bdat', encoding='latin1')

# récuperér le guide technique du questionnaire
# ce fichier inclut le nom de la variable, la question qu'elle encode, l'année du questionnaire
# mais également des notes pour la compréhension de la base de données initiale

cd.lecture_fichier(fs, f"{chemin_nsch}NSCH_Dictionary_11-29-2025_0359_PM.csv", f"data/nsch/NSCH_Dictionary_11-29-2025_0359_PM.csv")

# Element de précaution : GUIDE contient plusieurs index de meme nom
# mais les caractéristiques ne sont pas identiques

guide = pd.read_csv("data/nsch/NSCH_Dictionary_11-29-2025_0359_PM.csv")

In [3]:
# Récuperer les fichiers associés au shapefile - utiles pour les cartes (depuis le stockage S3 d'un membre du groupe)
# Stocker les fichiers dans un dossier temporel data/map

chemin_map = "inacampan/diffusion/Determinants_of_children-s_health/Map/"

for ext in ["shp", "shx", "dbf", "prj"]:
    cd.lecture_fichier(fs, f"{chemin_map}cb_2024_us_state_20m.{ext}", f"data/map/cb_2024_us_state_20m.{ext}")

# lecture du fichier
gdf = gpd.read_file("data/map/cb_2024_us_state_20m.shp")

## Description des données et statistiques descriptives

Conformément au site qui héberge les bases de données NSCH, le questionnaire de présélection (NSCH-S1) déterminait si l’adresse représentait une résidence occupée et s’il y avait des enfants admissibles âgés de 0 à 17 ans vivant à cette adresse échantillonnée.

Le questionnaire thématique ("topical") comprenait des questions détaillées portant sur un enfant sélectionné au hasard dans le ménage. Les ménages recevaient l’un des trois questionnaires thématiques spécifiques à l’âge, en fonction de l’âge de l’enfant échantillonné : 
* NSCH-T1 (ou T1) pour les enfants âgés de 0 à 5 ans,
* NSCH-T2 (ou T2) pour les enfants âgés de 6 à 11 ans, ou
* NSCH-T3 (ou T3) pour les enfants âgés de 12 à 17 ans.

Le type du questionnaire est codé dans la variable ```FORMTYPE```.
Dans notre projet nous regardons uniquement le questionnaire "topical". 

In [6]:
df = dfs["2023"]
df.describe()

,HEIGHT,TOTKIDS_R,TENURE,HHLANGUAGE,SC_AGE_YEARS,SC_SEX,K2Q35A_1_YEARS,BIRTH_MO,BIRTH_YR,K6Q41R_STILL,...,HHCOUNT_IF,HIGRADE,HIGRADE_TVIS,FPL_I1,FPL_I2,FPL_I3,FPL_I4,FPL_I5,FPL_I6,FWC
count,32319.000000,55162.000000,55162.000000,54862.000000,55162.000000,55162.000000,1971.000000,54770.000000,54675.000000,18118.000000,...,55162.000000,55162.000000,55162.000000,55162.000000,55162.000000,55162.000000,55162.000000,55162.000000,55162.000000,55162.000000
mean,151.687657,1.876690,1.455187,1.126135,8.339563,1.486422,4.868087,6.613255,2014.314092,1.884204,...,0.031761,2.829393,3.485606,287.280537,287.401998,287.531362,288.364091,287.652804,287.905080,1308.472653
std,20.522720,0.870582,0.787676,0.439835,5.303088,0.499820,3.478265,3.404391,5.324464,0.319990,...,0.175365,0.436074,0.799796,124.920459,124.878680,124.957976,124.475651,124.808909,124.524807,2334.642676
min,93.980000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,2005.000000,1.000000,...,0.000000,1.000000,1.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,11.006690
25%,134.620000,1.000000,1.000000,1.000000,4.000000,1.000000,2.000000,4.000000,2009.000000,2.000000,...,0.000000,3.000000,3.000000,179.000000,179.000000,179.000000,182.000000,179.000000,181.000000,263.179394
50%,154.940000,2.000000,1.000000,1.000000,8.000000,1.000000,3.000000,7.000000,2015.000000,2.000000,...,0.000000,3.000000,4.000000,337.000000,337.000000,337.000000,337.500000,337.000000,337.000000,639.147322
75%,167.630000,2.000000,2.000000,1.000000,13.000000,2.000000,6.000000,10.000000,2019.000000,2.000000,...,0.000000,3.000000,4.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,1416.809570
max,211.050000,4.000000,4.000000,3.000000,17.000000,2.000000,15.000000,12.000000,2023.000000,2.000000,...,1.000000,3.000000,4.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,54078.402188


In [32]:
# La base de données de 2023 est dense, avec un total de 452 de variables sur 55162 individus (à travers les états-Unis).
# Sur l'ensemble des enfants dans l'échantillons : 
# (i) 21524 sont âgés de 0 à 5 ans,
# (ii) 18397 sont âgés de 6 à 11 ans et 
# (iii) 15241 sont âgés de 11 à 17 ans.

print(df["FORMTYPE"].value_counts())
print(df["FIPSST"].value_counts()[:5])

FORMTYPE
T1    21524
T3    18397
T2    15241
Name: count, dtype: int64
FIPSST
06    4696
20    2805
27    2400
22    2093
39    1735
Name: count, dtype: int64


On regarde le pourcentage de valeurs manquantes par colonne, par ordre décroissant.
On observe des taux de valeurs manquantes très élevés pour certaines questions qui ne sont posées que dans des cas très spécifiques et qui dépendent d’une réponse précédente. (Cela est rassurant.)

Par exemple, pour la variable ```CYSFIB_SCREEN``` (avec un taux de 99.94% de valeurs manquantes), celle-ci encode la question :

**Was this condition identified through a blood test done shortly after birth? … These tests are sometimes called newborn screening.** Cette question est ignorée si ```CYSTFIB = 2```. La variables ```CYSFIB``` a un taux très très faible (0.19%) de valeurs manquantes et parmi les valeurs prises, ```CYSFIB!=2``` dans seulement 30 questionnaires.

Pour les variables ```DIABETES_DESC``` et ```DIABETES_CURR```, les questions qu'elles encodent sont ignorées dès lorsque ```DIABETES = 2``` (ce qui regroupe seulement 40 questionnaires).

In [6]:
missing_values = df.isna().mean().sort_values(ascending=False) * 100

print("Valeurs manquantes en pourcentage par colonne\n",
    missing_values)
print("Nombre de colonnes avec au moins 50% de valeurs manquantes\n",(missing_values >= 50).sum())

# Illustration de l'exemple de CYSTFIB et DIABETES
print("---------------------------------")
print("Informations sur la variable CYSTFIB_SCREEN : ", guide.loc["CYSTFIB_SCREEN","Type"], "\n",
    guide.loc["CYSTFIB", "Description"])
print("Valeurs manquantes pour la variable CYSTFIB : \n",
    (df["CYSTFIB"].isna().mean() * 100).round(2), "%")
print(df["CYSTFIB"].value_counts())

print("---------------------------------")
print("Valeurs manquantes pour la variable DIABETES : \n",
    (df["DIABETES"].isna().mean() * 100).round(2), "%")
print(df["DIABETES"].value_counts())

Valeurs manquantes en pourcentage par colonne
 CYSTFIB_SCREEN    99.949240
DIABETES_DESC     99.945615
DIABETES_CURR     99.922048
K2Q38C            99.789710
LIVEUSA_MO        99.778833
                    ...    
FPL_I3             0.000000
FPL_I4             0.000000
FPL_I5             0.000000
FPL_I6             0.000000
FWC                0.000000
Length: 456, dtype: float64
Nombre de colonnes avec au moins 50% de valeurs manquantes
 180
---------------------------------
Informations sur la variable CYSTFIB_SCREEN :  Skip if CYSTFIB=2 
 Header: Has a doctor or other health care provider EVER told you that this child has?...Cystic Fibrosis?
Valeurs manquantes pour la variable CYSTFIB : 
 0.19 %
CYSTFIB
2.0    55025
1.0       30
Name: count, dtype: int64
---------------------------------
Valeurs manquantes pour la variable DIABETES : 
 0.28 %
DIABETES
2.0    54966
1.0       40
Name: count, dtype: int64


### Choix des variables d'intérêt

Dans le guide, les variables sont classées dans plusieurs grandes catégories, en fonction de la question de recherche et également de la base de données associée. En vue de notre base de données, on peut garder uniquement les variables de type "Topical" et "Operational" (colonne "Variable").

In [30]:
guide = guide[guide["Variable"].isin({"Topical", "Operational"})]
print(guide["Source"].unique())

["J. About You/Child's Caregivers" 'Imputation Flag'
 'I. Your Family and Household' "A. This Child's Health"
 'C. Health Care Services' 'F. Providing Health Care'
 'G. Learning and Activities' 'H. About You and This Child'
 'D. Health Care Providers' 'B. This Child as an Infant'
 'Data Quality Flag' 'Geography' 'E. Health Insurance Coverage' 'Weight'
 'K. Household Information' 'Operational']
